In [13]:
import cv2 as cv
import numpy as np
import math

def img_rotate(fname, fname_pr):
    src = cv.imread(fname, cv.IMREAD_GRAYSCALE) 
    final = cv.imread(fname, cv.IMREAD_COLOR) 

    #очистка от шумов
    im_bw = cv.adaptiveThreshold(src, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 10)

    #изменение размера
    final_wide = 200 
    r = float(final_wide) / src.shape[1]
    dim = (final_wide, int(src.shape[0] * r))
    resized = cv.resize(im_bw, dim, interpolation = cv.INTER_AREA)

    #выделение границ Кэнни
    dst = cv.Canny(resized,400,100)

    cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR) 
    cdstP = np.copy(cdst) 

    #преобразование Хаффа
    lines = cv.HoughLines(dst, 1, np.pi / 120, 100, None, 0, 0)

    coner_array = [] 
    if lines is not None: 
        for i in range(0, len(lines)): 
            rho = lines[i][0][0] 
            theta = lines[i][0][1] 
            a = math.cos(theta) 
            b = math.sin(theta) 
            x0 = a * rho 
            y0 = b * rho 
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a))) 
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a))) 
            cv.line(cdst, pt1, pt2, (0,0,255), 1, cv.LINE_AA) 

        linesP = cv.HoughLinesP(dst, 1, np.pi / 180, 50, None, 20, 10)

        if linesP is not None: 
            for i in range(0, len(linesP)): 
                l = linesP[i][0] 
                cv.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 1, cv.LINE_AA) 
                coner_array.append(math.degrees(math.atan((l[3]-l[1])/(l[2]-l[0]))))

        coner=np.mean(coner_array) 
        (h, w) = src.shape[:2] 
        center = (w / 2, h / 2)  
        M = cv.getRotationMatrix2D(center, coner, 1.0) 
        binrotated = cv.warpAffine(src, M, (w, h),  borderMode=cv.BORDER_REPLICATE) 
        rotated = cv.warpAffine(final, M, (w, h),  borderMode=cv.BORDER_REPLICATE) 

        cv.imwrite(fname_pr, rotated) 